# Env

In [9]:
%%bash
git clone https://github.com/almaan/stereoscope 
cd stereoscope
./setup.py install

# Preprocess data

In [6]:
# load to stereoscope format

import pandas as pd
import numpy as np
from copy import deepcopy
import os

st_coords = pd.read_csv('../datasets/seqFISH+/Out_rect_locations.csv')
st_coords.columns = ['barcode','','','x','y']
st_coords = st_coords.drop('',axis=1)
st_counts_10000genes = pd.read_csv('../datasets/seqFISH+/Out_gene_expressions_10000genes.csv')
st_counts_10000genes = st_counts_10000genes.rename({'Unnamed: 0':'barcode'}, axis=1)
sc_counts = pd.read_csv('../datasets/seqFISH+/raw_somatosensory_sc_exp.txt',sep='\t')
sc_counts = sc_counts.set_index('cell_id')
sc_counts = sc_counts.T
sc_labels = pd.read_csv('../datasets/seqFISH+/somatosensory_sc_labels.txt',header=None)
sc_labels.columns = ['celltype']
celltype = list(set(sc_labels.celltype))
celltype_dict = dict(zip([x+1 for x in range(len(celltype))],celltype))
metacell_dict = dict(zip([str(x+1) for x in range(len(celltype))],celltype))
sc_labels['cluster'] = [celltype.index(x)+1 for x in sc_labels.celltype]
sc_labels['barcode'] = sc_counts.index

save_dir = '../datasets/data4stereoscope'
if os.path.isdir(save_dir)==False:
    os.mkdir(save_dir)

sc_counts_stereoscope = deepcopy(sc_counts)
sc_counts_stereoscope = sc_counts_stereoscope.reset_index()
sc_counts_stereoscope = sc_counts_stereoscope.rename({'index':'cell'},axis=1)
sc_counts_stereoscope.to_csv(f'{save_dir}/sc_counts.tsv',index=False,sep='\t')

sc_labels_stereoscope = deepcopy(sc_labels[['barcode','celltype']])
sc_labels_stereoscope = sc_labels_stereoscope.rename({'barcode':'cell','celltype':'bio_celltype'},axis=1)
sc_labels_stereoscope.to_csv(f'{save_dir}/sc_labels.tsv',index=False,sep='\t')

st_counts_stereoscope = deepcopy(st_counts_10000genes)
st_counts_stereoscope['index'] = st_coords.barcode
st_counts_stereoscope = st_counts_stereoscope.drop('barcode',axis=1)
st_counts_stereoscope = st_counts_stereoscope.set_index('index')
st_counts_stereoscope.to_csv(f'{save_dir}/st_coords_counts.tsv',sep='\t')

# Main

In [ ]:
# Please run in command line
stereoscope run --sc_cnt ../datasets/data4stereoscope/sc_counts.tsv \
--sc_labels ../datasets/data4stereoscope/sc_labels.tsv \
--st_cnt ../datasets/data4stereoscope/st_coords_counts.tsv \
-o ../seqFISH_10000_Result/stereoscope -n 5000 \
--gpu -stb 1000 -scb 1000